In [ ]:
"""
Dataset Preprocessing Notebook
===========================================

A cell‑oriented script (compatible with Jupyter Lab/Notebook or VS Code) that
combines *licit* and *fraud* wallet feature tables, cleans them, and writes a
single labelled dataset.

* Reads  **LICIT_WALLETS** and **CHAINABUSE_METRICS** csv files from the data
  directory (default `../../../data/data_with_features`).
* Adds a `class` column (`"licit"` or `"fraud"`).
* Reports and drops rows containing *any* missing values, with counts before
  and after.
* Merges the two DataFrames, drops non‑feature identifier/time columns,
  and persists the result to `../../../data/dataset_with_label/chainabuse/`.
* Uses **environment variables** `DATA_BASE_PATH` and `OUT_BASE_PATH` to
  override defaults.
* Designed to be re‑executed safely; the output directory is created if
  missing and existing files are overwritten.
"""

In [ ]:
# ----------------------------------------------------------------------
# Configuration – set paths & filenames here
# ----------------------------------------------------------------------
from __future__ import annotations

import os
from pathlib import Path

# Base paths (override by exporting env‑vars before starting Jupyter)
DATA_BASE_PATH = Path(os.getenv("DATA_BASE_PATH", "../../../data/data_with_features"))
OUT_BASE_PATH = Path(os.getenv("OUT_BASE_PATH", "../../../data/dataset_with_label"))

LICIT_FILE = DATA_BASE_PATH / "licit_wallets_cleaned.csv"
FRAUD_FILE = DATA_BASE_PATH / "chainabuse_metrics.csv"

OUT_DIR = OUT_BASE_PATH / "chainabuse"
OUT_FILE = OUT_DIR / "chainabuse_fraud_unknown_filtered_licit_processed.csv"

OUT_DIR.mkdir(parents=True, exist_ok=True)
print(f"📂 Output will be saved to: {OUT_FILE}")

In [ ]:
# ----------------------------------------------------------------------
# Imports (data & utility libs)
# ----------------------------------------------------------------------
import pandas as pd
from typing import Tuple

pd.set_option("display.max_columns", None)  # show all columns when printing

In [ ]:
# ----------------------------------------------------------------------
# Helper functions
# ----------------------------------------------------------------------

def report_missing(df: pd.DataFrame, name: str) -> None:
    """Print a summary table of missing value counts for *df* (rows > 0)."""
    nan_counts = df.isna().sum()
    nan_df = nan_counts[nan_counts > 0]
    if nan_df.empty:
        print(f"✅ {name}: no missing values\n")
    else:
        print(f"⚠️  {name}: missing values by column →")
        print(nan_df.sort_values(ascending=False).to_frame("nan_count"))
        print()


def load_and_tag_csv(path: Path, label: str) -> pd.DataFrame:
    """Load CSV at *path* and add a constant `class` column=*label*."""
    if not path.exists():
        raise FileNotFoundError(path)
    df = pd.read_csv(path)
    df["class"] = label
    return df

In [ ]:
# ----------------------------------------------------------------------
# Load licit wallets
# ----------------------------------------------------------------------
licit_df = load_and_tag_csv(LICIT_FILE, "licit")
print(f"🔹 Licit wallets loaded: {len(licit_df):,} rows, {licit_df.shape[1]} columns")
report_missing(licit_df, "licit_df")

# Drop rows with any NaNs
licit_df_clean = licit_df.dropna().reset_index(drop=True)
print(f"🧹 Licit after dropna: {len(licit_df_clean):,} rows (removed {len(licit_df) - len(licit_df_clean):,})\n")

In [ ]:
# ----------------------------------------------------------------------
# Load fraud wallets (Chainabuse)
# ----------------------------------------------------------------------
elliptic_fraud_df = load_and_tag_csv(FRAUD_FILE, "fraud")
print(f"🔸 Fraud wallets loaded: {len(elliptic_fraud_df):,} rows, {elliptic_fraud_df.shape[1]} columns")
report_missing(elliptic_fraud_df, "elliptic_fraud_df")

elliptic_fraud_clean = elliptic_fraud_df.dropna().reset_index(drop=True)
print(f"🧹 Fraud after dropna: {len(elliptic_fraud_clean):,} rows (removed {len(elliptic_fraud_df) - len(elliptic_fraud_clean):,})\n")

In [ ]:
# ----------------------------------------------------------------------
# Combine datasets
# ----------------------------------------------------------------------
combined_df = pd.concat([licit_df_clean, elliptic_fraud_clean], ignore_index=True)
print(f"🔗 Combined dataset: {len(combined_df):,} rows, {combined_df.shape[1]} columns")

# Columns that are identifiers or timestamps and not useful as features
DROP_COLUMNS = ["wallet", "wallet_last_tx_ts", "wallet_creation_ts"]
combined_df.drop(columns=[c for c in DROP_COLUMNS if c in combined_df.columns], inplace=True)
print(f"🗑️  Dropped columns: {DROP_COLUMNS}\nCurrent shape: {combined_df.shape}\n")


In [ ]:
# ----------------------------------------------------------------------
# Persist to CSV
# ----------------------------------------------------------------------
combined_df.to_csv(OUT_FILE, index=False)
print(f"💾 Saved merged dataset → {OUT_FILE.relative_to(Path.cwd())}\n")

# %% [markdown]
"""
### Next steps
* Inspect class balance: `combined_df['class'].value_counts()`
* Perform train/test split and feature scaling.
* Visualise feature distributions.
"""